In [5]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.4.0
True


In [6]:
device = torch.device("cuda")
x = torch.tensor([1.0, 2.0, 3.0], device=device)
print(f"Tensor on GPU: {x}")
print(f"GPU Tensor Sum: {x.sum()}")

Tensor on GPU: tensor([1., 2., 3.], device='cuda:0')
GPU Tensor Sum: 6.0


In [7]:
# KNN & Mutual Information Preprocessing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score
import pandas as pd
import random
import numpy as np

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Load the dataset
data = pd.read_csv('dataset/train_re.csv')

# Prepare features (X) and target (y)
X = data.drop(columns=['PCIAT-PCIAT_Total','sii'])
y = data['sii']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)

# Define the enhanced neural network
class EnhancedClassificationNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout_rate=0.5, negative_slope=0.01):
        super(EnhancedClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], num_classes)
        self.dropout = nn.Dropout(dropout_rate)
        self.activation = nn.LeakyReLU(negative_slope)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.activation(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

# Define Weighted Kappa Loss
class WeightedKappaLoss(nn.Module):
    def __init__(self, num_classes, weight_type="quadratic"):
        super(WeightedKappaLoss, self).__init__()
        self.num_classes = num_classes
        self.weight_type = weight_type
        self.weights = self._create_weights()

    def _create_weights(self):
        weights = torch.zeros((self.num_classes, self.num_classes))
        for i in range(self.num_classes):
            for j in range(self.num_classes):
                if self.weight_type == "linear":
                    weights[i, j] = 1 - abs(i - j) / (self.num_classes - 1)
                elif self.weight_type == "quadratic":
                    weights[i, j] = 1 - ((i - j) ** 2) / ((self.num_classes - 1) ** 2)
        return weights

    def forward(self, logits, targets):
        # Move weights to the same device as logits
        self.weights = self.weights.to(logits.device)

        probs = torch.softmax(logits, dim=1)
        conf_matrix = torch.zeros((self.num_classes, self.num_classes), device=logits.device)
        for i in range(self.num_classes):
            for j in range(self.num_classes):
                conf_matrix[i, j] = torch.sum((targets == i).float() * probs[:, j])
        conf_matrix = conf_matrix / conf_matrix.sum()
        P_o = torch.sum(conf_matrix * self.weights)
        row_sums = conf_matrix.sum(dim=1)
        col_sums = conf_matrix.sum(dim=0)
        E = torch.outer(row_sums, col_sums)
        P_e = torch.sum(E * self.weights)
        kappa_w = (P_o - P_e) / (1 - P_e + 1e-7)
        loss = 1 - kappa_w
        return loss


class HybridLoss(nn.Module):
    def __init__(self, alpha, num_classes, weight_type="quadratic"):
        super(HybridLoss, self).__init__()
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kappa_loss = WeightedKappaLoss(num_classes=num_classes, weight_type=weight_type)

    def forward(self, logits, targets):
        ce = self.ce_loss(logits, targets)
        kappa = self.kappa_loss(logits, targets)
        return self.alpha * ce + (1 - self.alpha) * kappa

# Hyperparameters
input_size = X_train.shape[1]
hidden_sizes = [128, 64, 32]
num_classes = len(y.unique())
dropout_rate = 0.5
learning_rate = 0.001
num_epochs = 300
batch_size = 32


# List to store results for each alpha
results = []

# Range of alpha values to test
alpha_values = [round(x * 0.1, 1) for x in range(11)]

# Loop through alpha values
for alpha in alpha_values:
    print(f"Testing alpha = {alpha}")
    
    # Reset model, loss, and optimizer
    model = EnhancedClassificationNN(input_size, hidden_sizes, num_classes, dropout_rate).to(device)
    criterion = HybridLoss(alpha=alpha, num_classes=num_classes, weight_type="quadratic")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # DataLoader for batching
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        true_labels = y_test_tensor.cpu().numpy()
        predicted_labels = predicted.cpu().numpy()
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    weighted_kappa = cohen_kappa_score(true_labels, predicted_labels, weights="quadratic")
    print(f"Alpha: {alpha}, Accuracy: {accuracy:.4f}, Weighted Kappa: {weighted_kappa:.4f}")
    
    # Store results
    results.append({
        "alpha": alpha,
        "accuracy": accuracy,
        "weighted_kappa": weighted_kappa
    })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display results
print(results_df)

# model = EnhancedClassificationNN(input_size, hidden_sizes, num_classes, dropout_rate).to(device)
# alpha = 0.2
# criterion = HybridLoss(alpha=alpha, num_classes=num_classes, weight_type="quadratic")

# # Training loop
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # DataLoader for batching
# train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# # Training the model
# for epoch in range(num_epochs):
#     model.train()
#     for batch_X, batch_y in train_loader:
#         # Ensure data is on the correct device
#         batch_X, batch_y = batch_X.to(device), batch_y.to(device)

#         # Forward pass
#         outputs = model(batch_X)
#         loss = criterion(outputs, batch_y)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


# # Evaluating the model
# model.eval()
# with torch.no_grad():
#     outputs = model(X_test_tensor)
#     _, predicted = torch.max(outputs, 1)
#     true_labels = y_test_tensor.cpu().numpy()
#     predicted_labels = predicted.cpu().numpy()

# # Validation metrics
# accuracy = accuracy_score(true_labels, predicted_labels)
# weighted_kappa = cohen_kappa_score(true_labels, predicted_labels, weights="quadratic")
# print(f"Accuracy: {accuracy:.4f}, Weighted Kappa (sklearn): {weighted_kappa:.4f}")



Testing alpha = 0.0
Alpha: 0.0, Accuracy: 0.5850, Weighted Kappa: 0.4517
Testing alpha = 0.1
Alpha: 0.1, Accuracy: 0.6033, Weighted Kappa: 0.4252
Testing alpha = 0.2
Alpha: 0.2, Accuracy: 0.5832, Weighted Kappa: 0.4381
Testing alpha = 0.3
Alpha: 0.3, Accuracy: 0.5978, Weighted Kappa: 0.4192
Testing alpha = 0.4
Alpha: 0.4, Accuracy: 0.5905, Weighted Kappa: 0.4260
Testing alpha = 0.5
Alpha: 0.5, Accuracy: 0.5887, Weighted Kappa: 0.3859
Testing alpha = 0.6
Alpha: 0.6, Accuracy: 0.5923, Weighted Kappa: 0.3852
Testing alpha = 0.7
Alpha: 0.7, Accuracy: 0.5868, Weighted Kappa: 0.3681
Testing alpha = 0.8
Alpha: 0.8, Accuracy: 0.6051, Weighted Kappa: 0.3693
Testing alpha = 0.9
Alpha: 0.9, Accuracy: 0.5868, Weighted Kappa: 0.3779
Testing alpha = 1.0
Alpha: 1.0, Accuracy: 0.5759, Weighted Kappa: 0.3589
    alpha  accuracy  weighted_kappa
0     0.0  0.585009        0.451738
1     0.1  0.603291        0.425250
2     0.2  0.583181        0.438057
3     0.3  0.597806        0.419156
4     0.4  0.5904

In [9]:
# Regression Preprocessing

In [ ]:



import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score
import pandas as pd
import random
import numpy as np

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Load the dataset
data = pd.read_csv('dataset/train_rf.csv')

# Prepare features (X) and target (y)
X = data.drop(columns=['PCIAT-PCIAT_Total','sii'])
y = data['sii']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)

# Define the enhanced neural network
class EnhancedClassificationNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout_rate=0.5, negative_slope=0.01):
        super(EnhancedClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], num_classes)
        self.dropout = nn.Dropout(dropout_rate)
        self.activation = nn.LeakyReLU(negative_slope)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.activation(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

# Define Weighted Kappa Loss
class WeightedKappaLoss(nn.Module):
    def __init__(self, num_classes, weight_type="quadratic"):
        super(WeightedKappaLoss, self).__init__()
        self.num_classes = num_classes
        self.weight_type = weight_type
        self.weights = self._create_weights()

    def _create_weights(self):
        weights = torch.zeros((self.num_classes, self.num_classes))
        for i in range(self.num_classes):
            for j in range(self.num_classes):
                if self.weight_type == "linear":
                    weights[i, j] = 1 - abs(i - j) / (self.num_classes - 1)
                elif self.weight_type == "quadratic":
                    weights[i, j] = 1 - ((i - j) ** 2) / ((self.num_classes - 1) ** 2)
        return weights

    def forward(self, logits, targets):
        # Move weights to the same device as logits
        self.weights = self.weights.to(logits.device)

        probs = torch.softmax(logits, dim=1)
        conf_matrix = torch.zeros((self.num_classes, self.num_classes), device=logits.device)
        for i in range(self.num_classes):
            for j in range(self.num_classes):
                conf_matrix[i, j] = torch.sum((targets == i).float() * probs[:, j])
        conf_matrix = conf_matrix / conf_matrix.sum()
        P_o = torch.sum(conf_matrix * self.weights)
        row_sums = conf_matrix.sum(dim=1)
        col_sums = conf_matrix.sum(dim=0)
        E = torch.outer(row_sums, col_sums)
        P_e = torch.sum(E * self.weights)
        kappa_w = (P_o - P_e) / (1 - P_e + 1e-7)
        loss = 1 - kappa_w
        return loss

class HybridLoss(nn.Module):
    def __init__(self, alpha, num_classes, weight_type="quadratic"):
        super(HybridLoss, self).__init__()
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kappa_loss = WeightedKappaLoss(num_classes=num_classes, weight_type=weight_type)

    def forward(self, logits, targets):
        ce = self.ce_loss(logits, targets)
        kappa = self.kappa_loss(logits, targets)
        return self.alpha * ce + (1 - self.alpha) * kappa

# Hyperparameters
input_size = X_train.shape[1]
hidden_sizes = [128, 64, 32]
num_classes = len(y.unique())
dropout_rate = 0.5
learning_rate = 0.001
num_epochs = 300
batch_size = 32

# List to store results for each alpha
results = []

# Range of alpha values to test
alpha_values = [round(x * 0.1, 1) for x in range(11)]

# Loop through alpha values
for alpha in alpha_values:
    print(f"Testing alpha = {alpha}")
    
    # Reset model, loss, and optimizer
    model = EnhancedClassificationNN(input_size, hidden_sizes, num_classes, dropout_rate).to(device)
    criterion = HybridLoss(alpha=alpha, num_classes=num_classes, weight_type="quadratic")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # DataLoader for batching
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        true_labels = y_test_tensor.cpu().numpy()
        predicted_labels = predicted.cpu().numpy()
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    weighted_kappa = cohen_kappa_score(true_labels, predicted_labels, weights="quadratic")
    print(f"Alpha: {alpha}, Accuracy: {accuracy:.4f}, Weighted Kappa: {weighted_kappa:.4f}")
    
    # Store results
    results.append({
        "alpha": alpha,
        "accuracy": accuracy,
        "weighted_kappa": weighted_kappa
    })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display results
print(results_df)


# Model, loss, and optimizer
# model = EnhancedClassificationNN(input_size, hidden_sizes, num_classes, dropout_rate).to(device)
# alpha = 0.2
# criterion = HybridLoss(alpha=alpha, num_classes=num_classes, weight_type="quadratic")

# # Training loop
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # DataLoader for batching
# train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# # Training the model
# for epoch in range(num_epochs):
#     model.train()
#     for batch_X, batch_y in train_loader:
#         # Ensure data is on the correct device
#         batch_X, batch_y = batch_X.to(device), batch_y.to(device)

#         # Forward pass
#         outputs = model(batch_X)
#         loss = criterion(outputs, batch_y)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


# # Evaluating the model
# model.eval()
# with torch.no_grad():
#     outputs = model(X_test_tensor)
#     _, predicted = torch.max(outputs, 1)
#     true_labels = y_test_tensor.cpu().numpy()
#     predicted_labels = predicted.cpu().numpy()

# # Validation metrics
# accuracy = accuracy_score(true_labels, predicted_labels)
# weighted_kappa = cohen_kappa_score(true_labels, predicted_labels, weights="quadratic")
# print(f"Accuracy: {accuracy:.4f}, Weighted Kappa (sklearn): {weighted_kappa:.4f}")



Testing alpha = 0.0
Alpha: 0.0, Accuracy: 0.5814, Weighted Kappa: 0.4565
Testing alpha = 0.1
Alpha: 0.1, Accuracy: 0.5850, Weighted Kappa: 0.4165
Testing alpha = 0.2
Alpha: 0.2, Accuracy: 0.5960, Weighted Kappa: 0.4198
Testing alpha = 0.3
Alpha: 0.3, Accuracy: 0.6088, Weighted Kappa: 0.4624
Testing alpha = 0.4
Alpha: 0.4, Accuracy: 0.6106, Weighted Kappa: 0.4179
Testing alpha = 0.5
Alpha: 0.5, Accuracy: 0.6088, Weighted Kappa: 0.4214
Testing alpha = 0.6
Alpha: 0.6, Accuracy: 0.6088, Weighted Kappa: 0.4351
Testing alpha = 0.7
Alpha: 0.7, Accuracy: 0.5996, Weighted Kappa: 0.4268
Testing alpha = 0.8
Alpha: 0.8, Accuracy: 0.6271, Weighted Kappa: 0.4476
Testing alpha = 0.9
Alpha: 0.9, Accuracy: 0.5941, Weighted Kappa: 0.3613
Testing alpha = 1.0
Alpha: 1.0, Accuracy: 0.5996, Weighted Kappa: 0.3954
    alpha  accuracy  weighted_kappa
0     0.0  0.581353        0.456517
1     0.1  0.585009        0.416479
2     0.2  0.595978        0.419789
3     0.3  0.608775        0.462353
4     0.4  0.6106